In [4]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

def get_dataset_from_testcase(run):
    return run[0:run.rfind('_')]


def get_dataset_size(dataset):
    if dataset == "osm" or dataset == "books":
        return 800_000_000
    else:
        return 200_000_000

runs = []
dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/join_all'
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    if not os.path.exists(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        run['dataset_size'] = get_dataset_size(run['dataset'])
        runs.append(run)

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/eps_study'
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    if not os.path.exists(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        run['dataset_size'] = get_dataset_size(run['dataset'])
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)

Index(['command', 'dataset', 'dataset_size', 'result.checksum',
       'result.duration_ns', 'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.index.epsilon', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


In [5]:
df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["ratio"] = df["spec.common_key"]
df["tput"] = (df["dataset_size"] / df["ratio"]) / (df["result.duration_ns"] / (1000000000))
df["algo"] = df["spec.algo_name"]
display(df["algo"].unique())
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["epsilon"] = df["spec.index.epsilon"]


array(['lsj_sampledflatpgm256', 'inlj_flatpgm4096', 'lsj_btree4096',
       'inlj_sampledflatpgm4096', 'inlj_flatpgm1024', 'hash_join',
       'inlj_pgm4096', 'sort_join', 'inlj_pgm1024', 'inlj_btree1024',
       'lsj_sampledflatpgm1024', 'inlj_btree256',
       'inlj_sampledflatpgm1024', 'lsj_pgm256', 'lsj_btree1024',
       'lsj_btree256', 'inlj_flatpgm256', 'lsj_flatpgm1024',
       'inlj_sampledflatpgm256', 'lsj_sampledflatpgm4096', 'inlj_pgm256',
       'lsj_pgm4096', 'lsj_pgm1024', 'lsj_flatpgm4096', 'lsj_flatpgm256',
       'inlj_btree4096', 'flatpgm256', 'flatpgm2048', 'flatpgm4096',
       'flatpgm1024', 'sj', 'flatpgm8192'], dtype=object)

In [6]:
rows = duckdb.sql("SELECT ratio, dataset, algo, avg(tput) as v FROM df WHERE algo='lsj_flatpgm256' OR algo='lsj_flatpgm2048' GROUP BY ratio, dataset,algo ORDER BY ratio").df()
rows.pivot(index='ratio', values='v', columns=['dataset', 'algo'])


dataset,normal,lognormal,uniform_sparse,wiki,fb,osm,uniform_dense,books
algo,lsj_flatpgm256,lsj_flatpgm256,lsj_flatpgm256,lsj_flatpgm256,lsj_flatpgm256,lsj_flatpgm256,lsj_flatpgm256,lsj_flatpgm256
ratio,,,,,,,,
1,2.399758e+07,2.552373e+07,2.392211e+07,4.989806e+07,2.391989e+07,1.629589e+07,2.365700e+07,7.640513e+06
10,1.707562e+07,1.633166e+07,1.570817e+07,1.876881e+07,1.636946e+07,1.108617e+07,1.593545e+07,5.085657e+06
100,6.263522e+06,6.413230e+06,6.138370e+06,7.972858e+06,6.037815e+06,4.721197e+06,6.572164e+06,2.241222e+06
1000,1.604043e+06,1.584487e+06,1.579772e+06,1.833758e+06,1.488961e+06,1.467569e+06,1.702927e+06,7.127977e+05


In [7]:
# Ratio = 1, epsilon vs duration across multiple threads
def plot_for_ratio(dataset):
    rows = duckdb.sql(
        " SELECT dataset, threads, ratio, epsilon, algo, AVG(duration_sec) as v FROM df"
        " WHERE (algo='lsj_flatpgm256' OR algo='lsj_flatpgm1024' OR algo='lsj_flatpgm4096')"
        f" AND dataset='{dataset}'"
        " GROUP BY dataset, ratio, threads, epsilon, algo"
        " ORDER BY ratio, threads, epsilon"
    ).df()
    return rows.pivot(index='threads', values='v', columns=['epsilon', 'ratio'])


In [9]:
# Ratio = 1, epsilon vs duration across multiple threads
display('fb', plot_for_ratio('fb'))
display('wiki', plot_for_ratio('wiki'))
display('osm', plot_for_ratio('osm'))
display('books', plot_for_ratio('books'))
display('udense', plot_for_ratio('uniform_dense'))
display('usparse', plot_for_ratio('uniform_sparse'))
display('normal', plot_for_ratio('normal'))
display('lognormal', plot_for_ratio('lognormal'))

'fb'

epsilon,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0
ratio,1,1,1,10,10,10,100,100,100,1000,1000,1000
threads,,,,,,,,,,,,
1,26.190664,28.362248,30.211240,4.088599,4.086847,4.564430,1.104157,1.045334,0.977828,0.299930,0.462068,0.495449
2,13.609405,14.452971,17.613453,2.065043,2.133508,2.490935,0.569503,0.521568,0.529227,0.173093,0.235491,0.267236
4,8.389189,8.784014,9.723148,1.468388,1.512390,1.483382,0.325730,0.298773,0.327492,0.089801,0.131088,0.155002
8,5.996413,6.155027,6.441893,0.888175,0.922249,0.978765,0.219654,0.220341,0.220129,0.096747,0.119502,0.122903
16,4.995718,5.025974,4.683901,0.645955,0.666504,0.706084,0.210582,0.218051,0.229434,0.175474,0.142004,0.144517


'wiki'

epsilon,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0
ratio,1,1,1,10,10,10,100,100,100,1000,1000,1000
threads,,,,,,,,,,,,
1,11.596428,12.720760,14.053581,3.341000,3.319841,3.565870,0.679447,0.743503,0.636850,0.216608,0.271517,0.283918
2,6.697459,6.743496,7.464118,2.122817,1.608226,1.976445,0.380171,0.342897,0.420474,0.131939,0.149033,0.158578
4,3.925824,4.075155,4.534856,1.125969,1.034591,1.362840,0.237696,0.247868,0.251605,0.078038,0.095496,0.110595
8,3.088491,2.884251,3.676284,0.751399,0.799041,0.861033,0.189785,0.180700,0.193569,0.086375,0.111525,0.096064
16,2.323467,2.393951,2.508316,0.596770,0.581947,0.596662,0.160427,0.174208,0.175645,0.110819,0.114311,0.128340


'osm'

...

epsilon,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0
ratio,1,1,1,10,10,10,100,100,100,1000,1000,1000
threads,,,,,,,,,,,,
1,26.451258,28.480467,31.031793,3.803966,4.197975,4.344114,1.036641,0.928614,1.046255,0.300270,0.374882,0.490189
2,13.582522,14.865525,15.948952,2.209445,2.278136,2.083011,0.519061,0.510280,0.477822,0.143971,0.200502,0.260620
4,8.728079,9.118900,9.102296,1.538869,1.554502,1.700529,0.347108,0.341920,0.347992,0.100758,0.124542,0.179709
8,5.988339,6.359518,6.513364,0.949918,0.944782,1.014439,0.216327,0.205237,0.222948,0.091861,0.111476,0.128403
16,4.899415,4.742733,5.490894,0.666544,0.675626,0.702158,0.206222,0.206226,0.193915,0.121154,0.136171,0.169378


'normal'

epsilon,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0
ratio,1,1,1,10,10,10,100,100,100,1000,1000,1000
threads,,,,,,,,,,,,
1,26.234829,28.096915,30.368441,3.561598,4.205500,4.586375,0.994933,0.785136,1.005801,0.289290,0.412675,0.438096
2,13.330235,14.845427,16.284912,1.874656,2.134439,2.419884,0.475625,0.427696,0.481612,0.147392,0.202019,0.240564
4,8.091567,8.874047,9.724246,1.470205,1.545345,1.665798,0.305228,0.304426,0.294460,0.091512,0.118788,0.143066
8,6.088394,6.344533,6.169768,0.837402,0.937439,1.002453,0.214783,0.216014,0.215562,0.090225,0.105144,0.119148
16,5.033523,5.309408,5.241919,0.640698,0.685556,0.698924,0.222017,0.189023,0.196822,0.138839,0.127609,0.142594


'lognormal'

epsilon,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0
ratio,1,1,1,10,10,10,100,100,100,1000,1000,1000
threads,,,,,,,,,,,,
1,24.927506,27.321879,29.471769,4.003646,4.365843,4.476430,1.002268,0.878553,0.984343,0.306410,0.390009,0.435149
2,12.715500,14.125942,15.090289,2.161790,2.321908,2.490459,0.516857,0.545615,0.515337,0.152004,0.194671,0.261948
4,8.066458,8.365460,9.157828,1.417272,1.356959,1.633166,0.310736,0.302459,0.308466,0.091847,0.124318,0.148240
8,5.502039,6.286209,6.376248,0.891186,0.912284,0.955612,0.209971,0.210362,0.253809,0.109578,0.111108,0.121531
16,4.701473,4.787259,5.263620,0.650854,0.676181,0.687381,0.198089,0.194152,0.199078,0.107273,0.151194,0.144077
